# Analyze multi-modal information

In [1]:
import os
import pandas as pd

data_path = '/vol/bitbucket/mh1022/temporal-modelling-icd/dataset/multimodal'
labevents_df = pd.read_csv(os.path.join(data_path, 'LABEVENTS.csv'))
microbiologyevents_df = pd.read_csv(os.path.join(data_path, 'MICROBIOLOGYEVENTS.csv'))
prescriptions_df = pd.read_csv(os.path.join(data_path, 'PRESCRIPTIONS.csv'))


/tmp/ipykernel_512271/166864307.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions_df = pd.read_csv(os.path.join(data_path, 'PRESCRIPTIONS.csv'))


In [9]:
d_labitems_df = pd.read_csv(os.path.join(data_path, 'D_LABITEMS.csv'))

### LABEVENTS

In [10]:
import numpy as np
# filter out rows with HADM_ID = NaN
labevents_df = labevents_df[~labevents_df['HADM_ID'].isna()]
# join with d_labitems_df to get the measurmeent name
# LABEL - concept rerpesented
# FLUID - fluid on which the measurement was performed
# CATEGORY - category of the measurement (higher-level information)
labevents_df = labevents_df.merge(d_labitems_df[['ITEMID', 'LABEL', 'FLUID','CATEGORY']], how='left', on='ITEMID')

labevents_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,LABEL,FLUID,CATEGORY
0,441,3,145834.0,50868,2101-10-20 16:40:00,17,17.0,mEq/L,NaN,Anion Gap,Blood,Chemistry
1,442,3,145834.0,50882,2101-10-20 16:40:00,25,25.0,mEq/L,NaN,Bicarbonate,Blood,Chemistry
2,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal,"Calcium, Total",Blood,Chemistry
3,444,3,145834.0,50902,2101-10-20 16:40:00,99,99.0,mEq/L,abnormal,Chloride,Blood,Chemistry
4,445,3,145834.0,50910,2101-10-20 16:40:00,48,48.0,IU/L,NaN,Creatine Kinase (CK),Blood,Chemistry


In [11]:
labevents_df['HADM_ID'].unique()

array([145834., 163353., 185777., ..., 175386., 167146., 184477.])

In [12]:
labevents_df['FLAG'].unique()

array([nan, 'abnormal', 'delta'], dtype=object)

In [13]:
labevents_df['CATEGORY'].unique()

array(['Chemistry', 'Hematology', 'BLOOD GAS', 'Blood Gas', 'CHEMISTRY',
       'HEMATOLOGY'], dtype=object)

In [14]:
labevents_df['FLUID'].unique()

array(['Blood', 'BLOOD', 'Urine', 'URINE', 'Ascites',
       'Cerebrospinal Fluid (CSF)', 'Other Body Fluid', 'Bone Marrow',
       'Pleural', 'Joint Fluid', 'Stool', 'OTHER BODY FLUID'],
      dtype=object)

In [16]:
labevents_df['LABEL'].nunique()

561

In [ ]:
# TODO: define a way to integrate w ith model: perhaps simply concatenate "Lab measurement of category {} and fluid {} with label {} was performed with result {}".format(category, fluid, label, flag)


 ### MICROBIOLOGY EVENTS

In [17]:
microbiologyevents_df.head()
# if org name org itemid is null it means no organism grew in the culture


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# filter on org item id not null
microbiologyevents_df = microbiologyevents_df[~microbiologyevents_df['ORG_ITEMID'].isna()]
microbiologyevents_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
14,758,101,175533,2196-09-28 00:00:00,2196-09-28 20:58:00,70079.0,URINE,80155.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",1.0,NaN,NaN,NaN,NaN,NaN,NaN
26,770,105,128744,2189-02-20 00:00:00,2189-02-20 23:30:00,70079.0,URINE,80223.0,PROBABLE ENTEROCOCCUS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
27,771,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90012.0,GENTAMICIN,<=0.5,<=,1.0,S
28,772,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90016.0,OXACILLIN,0.5,=,1.0,S


In [19]:
# get number of unique org item ids
microbiologyevents_df['ORG_ITEMID'].nunique()

309

In [20]:
### PRESCRIPTIONS
prescriptions_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO
